建物データと浸水面標高ラスターデータから建物の被災データを生成。（必要手続き：Driveの接続）

3_CalcFloodDEMRaster.ipynbの実行後は再起動させる必要があります。

In [ ]:
# @title Googleドライブの準備
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title 日本の調査IDと自治体コードから住所情報を得る
survey_id = "A002005212020" #@param {type: "string"}
pref_code = "40" #@param
city_code = "203" #@param

In [ ]:
# @title 設定するパラメータ

# ケース名：解析を通して使います。
casename = "kurume-r2"

maxdepth = 4.27

local_government_code = f"{pref_code}{city_code}"

# 確認用画像出力用フラグ
flg_checkDisp =  False


In [ ]:
# @title 詳細設定用パラメータ（基本的には操作不要）

enable_savefig = False # 図をGoogleドライブに保存する？

zoomlevel = 15 # DEM zoom level to fetch

# デバッグ用出力
global globalflag_debug

# 保存先
path_home = "/content/drive/MyDrive/plateau-2023-uc01-satellite-analytics/PLATEAU-FloodSAR/"
#path_cgml = path_home + "/CityGML/"
path_dem = path_home + "/DEM/"
path_case = path_home + casename + "/"
path_upload = path_case + "ForUpload/"
json_buildings = path_case+"buildings_parsed.json"
json_boundary = path_case +"boundary.json"
file_bbox = path_case + "boundbox.npy"
file_dem_fmt = path_case + "dem_{}_{}-{}_{}-{}.npz"

file_flooddem = path_case + "flood_dem_{:04d}.npz"
file_flooddepth = path_case + "flood_depth_{:04d}.npz"

path_heattile_tmp = "heat/"

file_building_csv = path_upload + "buildings.csv"
file_floodHeattile_yukauemokuzou_zip = path_upload+"heatmap_yukaue_mokuzou.zip"
file_floodHeattile_yukashitamokuzou_zip = path_upload+"heatmap_yukashita_mokuzou.zip"
file_floodHeattile_allmokuzou_zip = path_upload+"heatmap_all_mokuzou.zip"

In [ ]:
# @title shapeファイルをダウンロードする．
%cd {path_home}
!mkdir shapes
%cd shapes

import requests
import re
from urllib.parse import unquote

# ダウンロードするファイルのURL
local_government_code = f"{pref_code}{city_code}"
url = f"https://www.e-stat.go.jp/gis/statmap-search/data?dlserveyId={survey_id}&code={local_government_code}&coordSys=1&format=shape&downloadType=5&datum=2000"

def get_filename_from_cd(cd):
    """
    Content-Dispositionヘッダーからファイル名を取得する関数
    """
    if not cd:
        return None
    fname = re.findall("filename\*=UTF-8''(.+)", cd)
    if len(fname) == 0:
        return None
    return unquote(fname[0])

# ファイルをダウンロードし保存する関数
def download_file(url):
    """
    指定されたURLからファイルをダウンロードし、Content-Dispositionヘッダーから取得したファイル名で保存する
    """
    with requests.get(url, stream=True) as response:
        response.raise_for_status()
        filename = get_filename_from_cd(response.headers.get('content-disposition'))
        if not filename:
            filename = 'downloaded_file.zip'
        with open(filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"ファイルがダウンロードされました: {filename}")
    return filename

# ファイルのダウンロード
shp_filename = download_file(url)

import os
shape_dir = os.path.splitext(shp_filename)[0]
!mkdir {shape_dir}
!unzip -o {shp_filename} -d {shape_dir}

%cd {path_home}
from glob import glob

address_shp = glob(f"shapes/{shape_dir}/*.shp")[0] # このコードの場所からみた住所のshpファイルの相対パス
shape_file = path_home + address_shp

/content/drive/MyDrive/plateau-2023-uc01-satellite-analytics/PLATEAU-FloodSAR
mkdir: cannot create directory ‘shapes’: File exists
/content/drive/MyDrive/plateau-2023-uc01-satellite-analytics/PLATEAU-FloodSAR/shapes
ファイルがダウンロードされました: A002005212020DDSWC40203.zip
mkdir: cannot create directory ‘A002005212020DDSWC40203’: File exists
Archive:  A002005212020DDSWC40203.zip
  inflating: A002005212020DDSWC40203/r2ka40203.shp  
  inflating: A002005212020DDSWC40203/r2ka40203.shx  
  inflating: A002005212020DDSWC40203/r2ka40203.dbf  
  inflating: A002005212020DDSWC40203/r2ka40203.prj  
/content/drive/MyDrive/plateau-2023-uc01-satellite-analytics/PLATEAU-FloodSAR


In [ ]:
# @title 主要ライブラリインポート
!pip install git+https://github.com/eukarya-inc/plateauutils.git@main#egg=plateauutils
import json
import matplotlib.pyplot as plt
import numpy as np
from progressbar import progressbar
from shapely.geometry import MultiPolygon, shape, mapping, Point
!pip install rasterio
import rasterio
from rasterio.features import geometry_mask
import fiona
import csv
import sys
import shutil
from plateauutils.flood_converter.flood_to_png import FloodToPng
#sys.path.append(path_home)
import plateau_floodsar_lib as pfsl

  Cloning https://github.com/eukarya-inc/plateauutils.git (to revision main) to /tmp/pip-install-ra95z9r7/plateauutils_cb43a44035fd46d3bd9ee1ac7a6d6bb6
  Running command git clone --filter=blob:none --quiet https://github.com/eukarya-inc/plateauutils.git /tmp/pip-install-ra95z9r7/plateauutils_cb43a44035fd46d3bd9ee1ac7a6d6bb6
  Resolved https://github.com/eukarya-inc/plateauutils.git to commit 988d053eae5ff3b0bd9e23445fdb6171796a418f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://gitlab.com/Oslandia/py3dtiles.git (to revision 489289d2f7310fbcb8e95710e250d893999adf5d) to /tmp/pip-install-ra95z9r7/py3dtiles_0c68e063942949f0a24d1d21945748ba
  Running command git clone --filter=blob:none --quiet https://gitlab.com/Oslandia/py3dtiles.git /tmp/pip-install-ra95z9r7/py3dtiles_0c68e063942949f0a24d1d21945748ba
  Running command git rev-parse -q --verify 'sha^489289d2f7310fbcb8e95710e250d89399

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 30.7 MB/s eta 0:00:00


In [ ]:
# JSONファイルからデータを読み込む
with open(json_buildings, 'r') as f:
    buildings = json.load(f)

print(len(buildings))

102376


In [ ]:
print(buildings[0:10])
print(len([vv for vv in buildings if vv["measured_height"] <2.1]))

[{'gid': 'bldg_383f1804-aa34-4634-949f-f769e09fa92d', 'center': [130.41263587199947, 33.22489181671553], 'min_height': 3.805999994277954, 'measured_height': 9.3, 'building_structure_type': '非木造', 'dem': 4.211194651213488}, {'gid': 'bldg_877dea60-35d0-4fd9-8b02-852e39c75d81', 'center': [130.41619367090038, 33.22492719812357], 'min_height': 4.454999923706055, 'measured_height': 3.0, 'building_structure_type': '非木造', 'dem': 4.511449391826913}, {'gid': 'bldg_b75ca4b1-92b3-44a5-ad16-5e72cd09b0f2', 'center': [130.41286751062674, 33.22492397949454], 'min_height': 3.700000047683716, 'measured_height': 9.4, 'building_structure_type': '非木造', 'dem': 3.956987680630892}, {'gid': 'bldg_61b43d48-4425-4801-985c-f6189fa71458', 'center': [130.46168907863304, 33.22495868239884], 'min_height': 4.843999862670898, 'measured_height': 3.0, 'building_structure_type': '不明', 'dem': 4.896962536308122}, {'gid': 'bldg_d1818edb-3724-41ef-b0bb-53d428017daa', 'center': [130.46219544576414, 33.224922353136535], 'min_he

In [ ]:
building_types = set([vv["building_structure_type"] for vv in buildings])
print(building_types)

{'非木造', '鉄筋コンクリート造', '不明', None, '木造・土蔵造'}


In [ ]:
#depth_map = np.load(file_flooddepth.format(int(maxdepth*100)))
dem_map = np.load(file_flooddem.format(int(maxdepth*100)))

In [ ]:
#lons = depth_map["lons"]
#lats = depth_map["lats"]
#depthmap = depth_map["floodmap_depth"]
lons = dem_map["lons"]
lats = dem_map["lats"]
demmap = dem_map["floodmap_dem"]

In [ ]:
#np.set_printoptions(threshold=np.inf)
print(lons)
print(lats)

[130.37475586 130.37479877 130.37484169 ... 130.77013493 130.77017784
 130.77022076]
[33.41304731 33.41301145 33.41297559 ... 33.21116941 33.21113355
 33.21109768]


In [ ]:
print(demmap)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [ ]:
if flg_checkDisp:
  ax =plt.subplot(1,1,1)
  #img = ax.contourf(lons,lats, depthmap,levels=np.arange(0,maxdepth+0.1,0.25))
  img = ax.contourf(lons,lats, demmap)#,levels=np.arange(0,maxdepth+0.1,0.25))
  ax.axis("equal")
  plt.colorbar(img)
  #ax.set_xlim(130.4,130.5)
  #ax.set_ylim(33.22,33.32)

In [ ]:
bldgtype_yukaue_depth = {"木造・土蔵造":0.45, "レンガ造・コンクリートブロック造・石造":0.45, "不明":0.45}
bldgtype_mokuzou = ["木造・土蔵造"]
bldgtype_renga = [""]
bldgtype_block = ["レンガ造・コンクリートブロック造・石造"]
def get_floodcategory(bldg_type, depth):
  min_depth = bldgtype_yukaue_depth[bldg_type] if bldg_type in bldgtype_yukaue_depth else 0.0
  category = "非浸水"
  if depth is not None and depth >0:
    if depth > min_depth :
      if min_depth == 0:
        category = "浸水"
      else:
        category = "床上"
    elif depth > 0:
      category = "床下"
  if bldg_type in bldgtype_mokuzou:
    category += "（木造）"
  elif bldg_type in bldgtype_renga:
    category += "（レンガ造）"
  elif bldg_type == "不明" or bldg_type is None:
    category += "（不明）"
  else:
    category += "（非木造）"
  return category

In [ ]:
print(get_floodcategory("木造・土蔵造", 1) )
print(get_floodcategory("木造・土蔵造", 0.2) )
print(get_floodcategory("不明", 1) )
print(get_floodcategory("不明", 0.3) )
print(get_floodcategory("非木造", 1) )
print(get_floodcategory(None, -1) )
print(get_floodcategory(None, None) )

床上（木造）
床下（木造）
床上（不明）
床下（不明）
浸水（非木造）
非浸水（不明）
非浸水（不明）


In [ ]:
def calc_floatIdx_of_list(val:float, list):
  """
  Args:
      val (float): value which the method finds index for.
      list (array-like): must be ordered (ascending or descending)
  """
  try:
    if list[0] < list[-1]:
      idl = np.where(val > list)[0][-1]
    else:
      idl = np.where(val < list)[0][-1]
  except IndexError:
    print("IndexErrorCheck:", val, (list[0], list[-1]))
    return np.nan
  if idl == len(list) -1:
    return idl
  return idl + (val-list[idl])/(list[idl+1]-list[idl])
  #return (val - list[0])/(list[-1]-list[0]) * (len(list)-1)

def calc_interpval_of_list(idx:float, list):
  il = int(idx)
  ih = int(idx + 1)
  if ih >= len(list):
    return list[-1]
  dd = idx - il
  return (1-dd)*list[il] + dd*list[ih]

def calc_InterpVal_lonlat(lon, lat, data, lons, lats):
  #print("Check1: ", lon, lat)
  cx = calc_floatIdx_of_list(lon, lons)
  cy = calc_floatIdx_of_list(lat, lats)
  #print("Check: ", cx, cy, "\n")
  if np.any(np.isnan([cy, cx])):
    return np.nan
  xl = int(cx)
  yl = int(cy)
  xh = xl + 1
  dx = cx-xl
  yh = yl + 1
  dy = cy - yl
  if xh > len(lons)-1:
    xh = xl
  if yh > len(lats)-1:
    yh = yl
  val_ll = data[yl,xl]
  val_hl = data[yh,xl]
  val_lh = data[yl,xh]
  val_hh = data[yh, xh]
  return (1-dy)*(1-dx)*val_ll + (1-dy)*dx*val_lh + dy*(1-dx)*val_hl + dy*dx*val_hh

In [ ]:
print(lons[0], lons[-1])
print(lats[0], lats[-1])

130.374755859375 130.77022075653076
33.413047314283666 33.211097684352545


In [ ]:
#@title 住所付与の準備

# shpファイルのパス

# マルチポリゴンのバウンディングボックスを取得します。
minx, miny, maxx, maxy = lons.min(), lats.min(), lons.max(), lats.max() # [130.412497, 33.224723, 130.68759971, 33.40869805]
# ラスタライズのパラメータを設定します。
raster_height, raster_width = demmap.shape # (5376, 6656)

transform = rasterio.transform.from_origin(minx, maxy, (maxx-minx)/raster_width, (maxy-miny)/raster_height)

lookup_table = []

# shpファイルを開く
with fiona.open(shape_file, 'r') as src:
    with rasterio.MemoryFile() as memfile:
        with memfile.open(driver='GTiff',
                          height=raster_height,
                          width=raster_width,
                          count=1,
                          dtype='uint8',
                          crs=src.crs,
                          transform=transform) as dataset:
            combined_mask = np.empty((dataset.height, dataset.width))
            combined_mask[:] = np.nan
            for i, feature in enumerate(src):
                # 各フィーチャからジオメトリを取得し、shapelyオブジェクトに変換
                polygon = shape(feature['geometry'])
                mask = geometry_mask([mapping(polygon)],
                                      transform=dataset.transform,
                                      invert=True,
                                      out_shape=(dataset.height, dataset.width))
                combined_mask[mask] = i
                del mask

                # PREF_NAME 福岡県
                # CITY_NAME 久留米市
                # S_NAME 花畑三丁目
                pref_name = feature["properties"]["PREF_NAME"]
                city_name = feature["properties"]["CITY_NAME"]
                s_name = feature["properties"]["S_NAME"]
                lookup_table.append({
                    "id": i,
                    "address": s_name,
                    "vector": polygon
                })


#plt.imshow(combined_mask)

new_lons = np.linspace(lons.min(), lons.max(), raster_width)  # 経度
new_lats = np.linspace(lats.max(), lats.min(), raster_height)   # 緯度

def get_address_ids(lon, lat, combined_mask):
    #center_lon, center_lat = building['center']

    # 緯度経度からピクセル座標に変換
    #lons_to_pixel = np.searchsorted(lons, center_lon) - 1
    #lats_to_pixel = np.searchsorted(lats, center_lat) - 1
    lons_to_pixel = int(calc_floatIdx_of_list(lon, new_lons))
    lats_to_pixel = int(calc_floatIdx_of_list(lat, new_lats))

    # # 周囲のセルを調べるための正規分布を作成
    # x, y = np.meshgrid(np.arange(combined_mask.shape[1]), np.arange(combined_mask.shape[0]))
    # pos = np.dstack((x, y))
    # rv = multivariate_normal([lons_to_pixel, lats_to_pixel], [[1.22, 0], [0, 1.22]])
    # gaussian_mask = rv.pdf(pos)

    # # 正規分布からしきい値を超えるセルのIDを取得
    # # threshold = gaussian_mask.mean()
    # # ids = combined_mask[gaussian_mask > threshold]

    ids = combined_mask[lats_to_pixel-1:lats_to_pixel+2, lons_to_pixel-1:lons_to_pixel+2] # 中心と周囲8ピクセルを見る
    unique_ids = np.unique(ids[~np.isnan(ids)])  # np.nanを除外してユニークなIDを取得

    return unique_ids

def find_address(lon, lat, combined_mask):
      address_ids = get_address_ids(lon, lat, combined_mask)
      address = ""
      if len(address_ids) == 0:
          address = '不明'
      elif len(address_ids) == 1:
          address = lookup_table[int(address_ids[0])]['address']
      else:
          # 複数のIDがある場合の処理
          # lookup_tableのベクトルと照合して正しいIDを見つける処理
          center_point =  Point(lon, lat) # Point(building['center'][0], building['center'][1])
          possible_addresses = []

          for address_id in address_ids:
              for entry in lookup_table:
                  if entry['id'] == address_id:
                      polygon = shape(entry['vector'])
                      if polygon.contains(center_point):
                          possible_addresses.append(entry['address'])
                          break  # 最初にマッチしたアドレスでループを抜ける

          if possible_addresses:
              address = possible_addresses[0]  # 最初に見つかった住所を採用
          else:
              # どのポリゴンにも含まれなかったら周囲の住所を全て列挙
              address = " / ".join([entry['address'] for entry in lookup_table if entry['id'] in address_ids])
      return address

# 処理を実行
#buildings_with_address = assign_address(buildings, combined_mask)

In [ ]:
for bb in progressbar(buildings):
  lon, lat = bb["center"]
  #print(lon, lat)
  #depth = calc_InterpVal_lonlat(lon, lat, depthmap, lons, lats)
  dem = calc_InterpVal_lonlat(lon, lat, demmap, lons, lats)
  depth = dem - bb["min_height"] #  bb["dem"]
  bb["lon"] = bb["center"][0]
  bb["lat"] = bb["center"][1]
  #print(depth)
  bb["out"] = False
  bb["flood_category"] = get_floodcategory(bb["building_structure_type"], depth)
  bb["depth"] = depth
  if "非浸水" in bb["flood_category"]:
    bb["out"] = True
    bb["depth"] = None
  bb["address"] = find_address(lon,lat,combined_mask)

100% (102376 of 102376) |################| Elapsed Time: 0:00:17 Time:  0:00:17


In [ ]:
print("2.1m未満：", len([bb for bb in buildings if bb["measured_height"] < 2.1]))
print([bb for bb in buildings if "非浸水" in bb["flood_category"]][0:10])

2.1m未満： 0
[{'gid': 'bldg_383f1804-aa34-4634-949f-f769e09fa92d', 'center': [130.41263587199947, 33.22489181671553], 'min_height': 3.805999994277954, 'measured_height': 9.3, 'building_structure_type': '非木造', 'dem': 4.211194651213488, 'lon': 130.41263587199947, 'lat': 33.22489181671553, 'out': True, 'flood_category': '非浸水（非木造）', 'depth': None, 'address': '城島町江上本'}, {'gid': 'bldg_877dea60-35d0-4fd9-8b02-852e39c75d81', 'center': [130.41619367090038, 33.22492719812357], 'min_height': 4.454999923706055, 'measured_height': 3.0, 'building_structure_type': '非木造', 'dem': 4.511449391826913, 'lon': 130.41619367090038, 'lat': 33.22492719812357, 'out': True, 'flood_category': '非浸水（非木造）', 'depth': None, 'address': '城島町江上'}, {'gid': 'bldg_b75ca4b1-92b3-44a5-ad16-5e72cd09b0f2', 'center': [130.41286751062674, 33.22492397949454], 'min_height': 3.700000047683716, 'measured_height': 9.4, 'building_structure_type': '非木造', 'dem': 3.956987680630892, 'lon': 130.41286751062674, 'lat': 33.22492397949454, 'out': T

In [ ]:
num_flooded = len([bb for bb in buildings if not bb["out"]])
num_yukashita = len([bb for bb in buildings if "床下" in bb["flood_category"]])

print(num_flooded,"(",num_yukashita,")/",len(buildings))

3018 ( 893 )/ 102376


In [ ]:
if flg_checkDisp:
  xx = [bb["center"][0] for bb in buildings]
  yy = [bb["center"][1] for bb in buildings]
  cc = ["orangered" if not bb["out"] else "purple" for bb in buildings]
  ss = [1 if not bb["out"] else 0.1 for bb in buildings]
  aa = [1 if not bb["out"] else 0.01 for bb in buildings]
  ax =plt.subplot(1,1,1)
  #img = ax.contourf(lons,lats, depthmap,levels=np.arange(0,maxdepth+0.1,0.25))
  img = ax.contourf(lons,lats, demmap)#,levels=np.arange(0,maxdepth+0.1,0.25))
  plt.scatter(xx,yy,c=cc,s=ss)#, alpha=aa)
  ax.axis("equal")
  plt.xlim([130.50, 130.53])
  plt.ylim([33.32, 33.34])
  plt.colorbar(img)
  plt.scatter

In [ ]:
!mkdir {path_upload}
csv_keylist = ['gid', 'lon', 'lat', 'min_height', 'flood_category','depth', 'address']
csv_header = ['gid', 'lon', 'lat', 'dem', 'flood_category','flood_depth', 'address']
with open(file_building_csv, "w") as ofile:
  ofile.write(",".join(f"'{w}'" for w in csv_header)+"\n")
  for bb in buildings:
    tempstr = ""
    for kk in csv_keylist:
      if isinstance(bb[kk], str):
        tempstr += f"'{bb[kk]}',"
      else:
        tempstr += f"{bb[kk]},"
    ofile.write(tempstr[:-1]+"\n")
    #print(tempstr[:-1])


mkdir: cannot create directory ‘/content/drive/MyDrive/plateau-2023-uc01-satellite-analytics/PLATEAU-FloodSAR/kurume-r2/ForUpload/’: File exists


In [ ]:
print(lons,lats)
print(lons[-1])

[130.37475586 130.37479877 130.37484169 ... 130.77013493 130.77017784
 130.77022076] [33.41304731 33.41301145 33.41297559 ... 33.21116941 33.21113355
 33.21109768]
130.77022075653076


In [ ]:
def dump2heatmap(lons, lats, buildings):
  gauss_hl = 63
  xx, yy = np.meshgrid(np.arange(-gauss_hl,gauss_hl+1),np.arange(-gauss_hl,gauss_hl+1))
  #gauss = 1/(2045*np.pi)*np.exp(-(xx**2+yy**2)/2048) # sigma = 32 px
  gauss = np.sqrt((xx**2+yy**2)) <= 31 #gauss_hl
  #print(np.shape(gauss))
  maxx = len(lons)
  maxy = len(lats)
  heatmap = np.zeros((maxy, maxx))
  for bb in buildings:
    if not bb["out"]:
      lon, lat = bb["center"]
      idx = int(calc_floatIdx_of_list(lon, lons))
      idy = int(calc_floatIdx_of_list(lat, lats))
      gxl, gxh = 0, gauss_hl*2 + 1
      gyl, gyh = 0, gauss_hl*2 + 1
      idxl, idxh = idx-gauss_hl, idx+gauss_hl+1
      idyl, idyh = idy-gauss_hl, idy+gauss_hl+1
      if idxl < 0:
        gxl = - idxl
        idxl = 0
      if idxh > maxx:
        gxh = idxh - maxx
        idxh = maxx
      if idyl < 0:
        gyl = - idyl
        idyl = 0
      if idyh > maxy:
        gyh = idyh - maxy
        idyh = maxy
      #print(idyl,idyh,idxl,idxh,gyl,gyh,gxl,gxh)
      heatmap[idyl:idyh,idxl:idxh] += gauss[gyl:gyh,gxl:gxh]
  return heatmap/(np.sum(gauss)*16) * 10000

In [ ]:
yukaue_mokuzou = dump2heatmap(lons, lats, [bb for bb in buildings if bb["flood_category"]=="床上（木造）"])
yukashita_mokuzou = dump2heatmap(lons, lats, [bb for bb in buildings if bb["flood_category"]=="床下（木造）"])
all_mokuzou = dump2heatmap(lons, lats, [bb for bb in buildings if "（木造）" in bb["flood_category"]])

In [ ]:
print(np.nansum(yukaue_mokuzou))
print(np.nansum(yukashita_mokuzou))
print(np.nansum(all_mokuzou))


525624.9999999987
525625.0000000003
1051249.9999999995


In [ ]:
if flg_checkDisp:
  chech_heatmap = all_mokuzou
  ax =plt.subplot(1,1,1)
  plt_dens = np.copy(chech_heatmap)
  plt_dens[chech_heatmap==0] = np.nan
  img = ax.pcolormesh(lons,lats, plt_dens)
  ax.axis("equal")
  plt.xlim([130.50, 130.53])
  plt.ylim([33.32, 33.34])
  plt.colorbar(img)

In [ ]:
if flg_checkDisp:
  ax =plt.subplot(1,1,1)
  plt_cat = np.copy(category)
  plt_cat[category==0] = np.nan
  img = ax.pcolormesh(lons,lats, plt_cat)
  ax.axis("equal")
  plt.xlim([130.50, 130.53])
  plt.ylim([33.32, 33.34])
  plt.colorbar(img)

In [ ]:
#@title XYZタイル分けされた点郡NPZファイルの生成関数
#print(len(lats)/256.0, len(lons)/256.0)

def save_tile_npz(lats,lons,map, classificationmap, zoom=15, dst_dir ="temp/"):
  dir_zoom = dst_dir + f"{zoom}/"
  if not os.path.exists(dir_zoom):
    os.makedirs(dir_zoom)
  res_lons = np.array([])
  res_lats = np.array([])
  res_maps = np.array([])
  res_class = np.array([])
  print(f" / total loop num: {len(lats)/256}")
  for jj, tmp_lat in progressbar(enumerate(lats[::256])):
    idj = jj*256
    sublats = lats[idj:idj+256]
    for ii, tmp_lon in enumerate(lons[::256]):
      idi = ii*256
      sublons = lons[idi:idi+256]
      xx,yy  = pfsl.calc_xyz_from_lonlat(tmp_lon, tmp_lat, zoom)
      #print(ii,jj, xx, yy)
      submap = map[idj:idj+256,idi:idi+256]
      subclass = classificationmap[idj:idj+256,idi:idi+256]
      grd_lons, grd_lats  = np.meshgrid(sublons,sublats)
      flt_lons = grd_lons.ravel()
      flt_lats = grd_lats.ravel()
      flt_map = submap.ravel()
      flt_class = subclass.ravel()
      #selected = [[lon,lat,dem] for lon, lat, dem in zip(flt_lons, flt_lats,flt_map) if not np.isnan(dem)]
      #print(flt_lons[0:5])
      #print(flt_lats[0:5])
      #print(flt_map[0:5])
      #print(ii, jj, f"{subzoom}-{xx}-{yy}", f"{subzoom}-{chk_tile[0]}-{chk_tile[1]}")
      if not os.path.exists(f"{dir_zoom}{xx}"):
        os.mkdir(f"{dir_zoom}{xx}")
      np.savez(f"{dir_zoom}{xx}/{yy}.npz", lons=flt_lons, lats=flt_lats, dem=flt_map, classification=flt_class)
      res_lons = np.append(res_lons, flt_lons)
      res_lats = np.append(res_lats, flt_lats)
      res_maps = np.append(res_maps, flt_map)
      res_class = np.append(res_class, flt_class)
  return res_lons, res_lats, res_maps, res_class


In [ ]:
worklist = [
  [yukaue_mokuzou, file_floodHeattile_yukauemokuzou_zip],
  [yukashita_mokuzou, file_floodHeattile_yukashitamokuzou_zip],
  [all_mokuzou, file_floodHeattile_allmokuzou_zip]
]
for heatmap, savefile in worklist:
  print(f"for {savefile}")
  #print(np.nansum(heatmap))
  category = np.zeros_like(heatmap)
  levels = [5, 10, 15, 20, 25]
  for ii, ll in enumerate(levels[::-1]):
    category[heatmap<ll] = len(levels)-ii
  category[heatmap==0] = 0
  category[heatmap>levels[-1]] = len(levels)+1
  heatmap[heatmap==0] = np.nan
  if os.path.exists("/heat_tmp"):
    !rm -rf /heat_tmp
  test_lons, test_lats, test_maps, test_class = save_tile_npz(lats,lons,heatmap,category,dst_dir="/heat_tmp/")
  if os.path.exists("/heattile_tmp/"):
    !rm -rf /heattile_tmp
  p = FloodToPng("/heat_tmp/")
  p.parse("/heattile_tmp/")
  shutil.make_archive(savefile[:-4], 'zip', "/heattile_tmp/")
  #!rm -rf "heat_tmp"
  #!rm -rf "heattile_tmp"

for /content/drive/MyDrive/plateau-2023-uc01-satellite-analytics/PLATEAU-FloodSAR/kurume-r2/ForUpload/heatmap_yukaue_mokuzou.zip


/ |#                                                  | 0 Elapsed Time: 0:00:00

 / total loop num: 22.0


| |              #                                   | 21 Elapsed Time: 0:04:48


for /content/drive/MyDrive/plateau-2023-uc01-satellite-analytics/PLATEAU-FloodSAR/kurume-r2/ForUpload/heatmap_yukashita_mokuzou.zip


...

| |                                        #         | 21 Elapsed Time: 0:04:34


for /content/drive/MyDrive/plateau-2023-uc01-satellite-analytics/PLATEAU-FloodSAR/kurume-r2/ForUpload/heatmap_all_mokuzou.zip


/ |#                                                  | 0 Elapsed Time: 0:00:00

 / total loop num: 22.0


| |                              #                   | 21 Elapsed Time: 0:04:26
